In [1]:
import numpy as np
import pandas as pd

In [2]:
d = {'Clubes': [], 'PG': [], 'j': [], 'V': [], 'E': [], 'D': [], 'GP': [], 'GC': [], 'SG': []}
dfClassChampionship = pd.DataFrame(data=d);
dfDataChampionship = pd.read_excel("b2003.xlsx", encoding='utf8')

In [3]:
dfDataChampionship.head()

,Horario,Dia,Data,Clube 1,Clube 2,Placar,Vencedor,Rodada,Arena
0,16h00,Sábado,29/03/2003,Guarani,Vasco,4x2,Guarani,1ª Rodada,Brinco de Ouro
1,16h00,Sábado,29/03/2003,Atlético-PR,Grêmio,2x0,Atlético-PR,1ª Rodada,Arena da Baixada
2,16h00,Domingo,30/03/2003,Flamengo,Coritiba,1x1,-,1ª Rodada,Maracanã
3,16h00,Domingo,30/03/2003,Goiás,Paysandu,2x2,-,1ª Rodada,Serra Dourada
4,16h00,Domingo,30/03/2003,Internacional,Ponte Preta,1x1,-,1ª Rodada,Beira Rio


** Get all clubes for championship **



In [4]:
#Cancatenando as colunas Clube 1 e Clube 2
clubs  =  pd.concat([dfDataChampionship['Clube 1'].str.lower(), dfDataChampionship['Clube 2'].str.lower()], axis=1, keys=['Clubes'])

In [5]:
#Obtem todos os clubes 
cb = pd.DataFrame({'Clubes': clubs['Clubes'].unique()})

In [6]:
#Todos os clubes que participaram dp campeonato
cb

,Clubes
0,guarani
1,atlético-pr
2,flamengo
3,goiás
4,internacional
5,criciúma
6,juventude
7,fortaleza
8,cruzeiro
9,vitória


In [7]:
def getNumberWinner(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter = df["Vencedor"].str.lower() == clube
    return (data[filter]['Vencedor'].count()).astype(np.int64)

def getNumberDepartures(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = df["Clube 1"].str.lower() == clube
    filter2 = df["Clube 2"].str.lower() == clube
    return (data[filter1]['Clube 1'].count() + data[filter2]['Clube 2'].count()).astype(np.int64)

def getPoints(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = df["Clube 1"].str.lower() == clube
    filter2 = df["Clube 2"].str.lower() == clube
    filter3 = df["Vencedor"].str.lower() == clube
    filter4 = (df["Clube 1"].str.lower() == clube) | (df["Clube 2"].str.lower() == clube)
    filter5 = df["Vencedor"].str.lower() == '-'
    
    v1 = data[(filter1) & (filter3)]
    v1 = v1['Vencedor'].count()
    v2 = data[(filter2) & (filter3)]
    v2 = v2['Vencedor'].count() 
    v3 = data[(filter4) & (filter5)]
    v3 = v3['Vencedor'].count()
    return ((v2*3)+(v1*3)+v3).astype(np.int64)

def getDraw(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = (df["Clube 1"].str.lower() == clube) | (df["Clube 2"].str.lower() == clube)
    filter2 = df["Vencedor"].str.lower() == '-'
    df = data[(filter1) & (filter2)]
    empates = df['Vencedor'].count()
    return empates.astype(np.int64)

def getDefeats(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = (df["Clube 1"].str.lower() == clube) | (df["Clube 2"].str.lower() == clube)
    filter2 = (df["Vencedor"].str.lower() != clube) & (df["Vencedor"].str.lower() != '-')
    df = data[(filter1) & (filter2)]
    derrotas = df['Vencedor'].count()
    return derrotas.astype(np.int64)

def getGP(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1  =  df["Clube 1"].str.lower() == clube
    filter2  =  df["Clube 2"].str.lower() == clube
    df1      =  data[(filter1)]
    df2      =  data[(filter2)]
    placar1  =  df1['Placar'].str.split('x')
    placar2  =  df2['Placar'].str.split('x')
    gp       =  0
    gc       =  0
    for g1, g2 in placar1:
        gp = (gp + pd.to_numeric( g1 ))
        gc = (gc + pd.to_numeric( g2 ))
            
    for g1, g2 in placar2:
        gp = (gp + pd.to_numeric( g2 )) 
        gc = (gc + pd.to_numeric( g1 ))
    return gp, gc

In [37]:
dfTable = cb[["Clubes"]].copy()
for column in ["PG", "J", "V", "E", "D", "GP", "GC", "SG"]:
    dfTable[column] = pd.np.nan
    
    
for index, row in dfTable.iterrows():
    c1  =  row['Clubes']
    if type(c1) != unicode:
        c1 =  c1.decode('utf-8')
    c1 =  c1.strip()
    
    pg       =  getPoints(dfDataChampionship, c1 )
    j        =  getNumberDepartures(dfDataChampionship, c1 )
    v        =  getNumberWinner(dfDataChampionship, c1 )
    e        =  getDraw(dfDataChampionship, c1 )
    d        =  getDefeats(dfDataChampionship, c1 )
    gp, gc   =  getGP(dfDataChampionship, c1 )
    
    if c1 == u'ponte preta':
        pg  -= 4  #Perdeu 4 pontos, 1 para o internacional, 3 para o juventude
        pg  += 3  #Recebeu 3 pontos devido o paysandu ter sido punido
    elif c1 == u'internacional':
        pg  += 1
        
        #Devido a um bug maluco no site oficial de onde coleto esses dados, 
        #tive que atribuir esse valor, pois não conseguir encontrar erros nos dados e 
        #todo restante dos dados batem com o do site oficial, exceto a pontuação.
        pg  += 1  
                  
    elif c1 == u'juventude':
        pg  += 3
    elif c1 == u'paysandu':
        pg  -= 8
    elif c1 == u'são caetano':
        pg  += 3  #Recebeu 3 pontos devido a punição do paysandu
    elif c1 == u'corinthians':
        pg  += 2  #Recebeu 2 pontos devido a punição do paysandu
    elif c1 == u'fluminense':
        pg  += 2  #Recebeu 2 pontos devido a punição do paysandu            
        
    dfTable.at[index, 'PG'] = pg
    dfTable.at[index, 'J']  = j
    dfTable.at[index, 'V']  = v
    dfTable.at[index, 'E']  = e
    dfTable.at[index, 'D']  = d
    dfTable.at[index, 'GP'] = gp
    dfTable.at[index, 'GC'] = gc
    dfTable.at[index, 'SG'] = gp - gc
    
    #print c1

dfTable

,Clubes,PG,J,V,E,D,GP,GC,SG
0,guarani,61.0,46.0,17.0,10.0,19.0,64.0,72.0,-8.0
1,atlético-pr,61.0,46.0,17.0,10.0,19.0,66.0,72.0,-6.0
2,flamengo,66.0,46.0,18.0,12.0,16.0,66.0,73.0,-7.0
3,goiás,65.0,46.0,18.0,11.0,17.0,78.0,63.0,15.0
4,internacional,72.0,46.0,20.0,10.0,16.0,59.0,57.0,2.0
5,criciúma,60.0,46.0,17.0,9.0,20.0,57.0,68.0,-11.0
6,juventude,53.0,46.0,12.0,14.0,20.0,55.0,70.0,-15.0
7,fortaleza,49.0,46.0,12.0,13.0,21.0,58.0,74.0,-16.0
8,cruzeiro,100.0,46.0,31.0,7.0,8.0,102.0,47.0,55.0
9,vitória,56.0,46.0,15.0,11.0,20.0,50.0,64.0,-14.0


In [38]:
reorderlist = [ 'Cruzeiro', 'Santos', 'São Paulo', 'São Caetano', 'Coritiba', 'Internacional', 
                'Atlético-mg', 'Flamengo', 'Goiás', 'Paraná', 'Figueirense', 'Atlético-pr', 'Guarani', 
                'Criciúma', 'Corinthians', 'Vitória', 'Vasco', 'Juventude', 'Fluminense', 'Grêmio', 
                'Ponte Preta', 'Paysandu', 'Fortaleza', 'Bahia']
listIndex   = []

#Processa a lista para gerar a lista de ordenação
for clube in reorderlist:
    for index, row in dfTable.iterrows():
        #Verifica se o valor de clube é um unicode, se não, convete isso.
        if type(clube) != unicode:
            c =  clube.decode('utf-8')
        else:
            c =  clube
        
        c1  =  row['Clubes']
        #print c1
        if type(c1) != unicode:
            c1 =  c1.decode('utf-8')
        
        c1 =  c1.strip()
        c  =  c.lower()
        
        
        
        #Verifica se o clube iterado é igual ao clube da lista, se sim
        #retorna seu indice
        if c1 == c:
            listIndex.append( index )
            break
            
#Reordena a lista
dfTable  =  dfTable.reindex(index=listIndex, columns=["Clubes", "PG", "J", "V", "E", "D", "GP", "GC", "SG"])
#Com a lista ordenada, corrige os indices.
dfTable.index =  np.arange(1, len(dfTable)+1)
dfTable

,Clubes,PG,J,V,E,D,GP,GC,SG
1,cruzeiro,100.0,46.0,31.0,7.0,8.0,102.0,47.0,55.0
2,santos,87.0,46.0,25.0,12.0,9.0,93.0,60.0,33.0
3,são paulo,78.0,46.0,22.0,12.0,12.0,81.0,67.0,14.0
4,são caetano,74.0,46.0,19.0,14.0,13.0,53.0,37.0,16.0
5,coritiba,73.0,46.0,21.0,10.0,15.0,67.0,58.0,9.0
6,internacional,72.0,46.0,20.0,10.0,16.0,59.0,57.0,2.0
7,atlético-mg,72.0,46.0,19.0,15.0,12.0,76.0,62.0,14.0
8,flamengo,66.0,46.0,18.0,12.0,16.0,66.0,73.0,-7.0
9,goiás,65.0,46.0,18.0,11.0,17.0,78.0,63.0,15.0
10,paraná,65.0,46.0,18.0,11.0,17.0,85.0,75.0,10.0
